<div class="alert alert-block" style="border: 2px solid #1976D2;background-color:#E3F2FD;padding:5px;font-size:0.9em;">
본 자료는 저작권법 제25조 2항에 의해 보호를 받습니다. 본 컨텐츠 및 컨텐츠 일부 문구등을 외부에 공개, 게시하지 말아주세요.<br>
본 강의를 잘 정리하면, 데이터 분석과 데이터 과학(머신러닝, 인공지능) 모두 가능합니다!<br>
<b><a href="https://school.fun-coding.org/">잔재미코딩</a> 에서 본 강의 기반 최적화된 로드맵도 확인하실 수 있습니다</b>
</div>

### 0. Get data
- https://www.kaggle.com/c/bike-sharing-demand

### 1. train/test 데이터 임포트

In [ ]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

import string
import warnings
import missingno
warnings.filterwarnings('ignore')

In [ ]:
df_train = pd.read_csv('bikesharing/train.csv')
df_test = pd.read_csv('bikesharing/test.csv')
df_all = pd.concat((df_train, df_test)).reset_index(drop=True)

In [ ]:
df_all.head()

### 자전거 공유 플랫폼

- 자전거 공유 플랫폼은 도시 곳곳에 설치한 키오스크를 통해서, 특정 장소에서 빌려서, 자전거를 타고, 다시 특정 장소에 리턴해놓는 시스템을 의미함

### 주요 컬럼

* datetime: 연-월-일 시:분:초
* season:
   - 1 = spring
   - 2 = summer
   - 3 = fall
   - 4 = winter 
* holiday: 해당 날짜(datetime)가 공휴일이었는지 여부(1은 공휴일)
   - 공휴일이 아닌, 일반적인 주말(일안하는 날)은 0으로 표기됨
* workingday: 해당 날짜(datetime)가 일하는 날(즉, 워킹데이, 공휴일 또는 주말이 아닌 날)인지 여부(1은 해당 날짜가 일하는 날임을 의미함)
* weather:
    * 1: Clear, Few clouds, Partly cloudy, Partly cloudy
    * 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
    * 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
    * 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog 
* temp: 절대 온도 (화씨, C)
* atemp: 체감 온도 (화씨, C)
* humidity: 상대 습도
* windspeed: 풍속
* casual: 등록안한 사람의 대여 횟수
* registered: 등록자의 대여 횟수
* count: 총 대여 횟수 (casual + registered)

### holiday 와 workingday 의 이해
- 2011-01-17 과 workingday 추이를 보면 holiday 와 workingday 를 이해할 수 있음

In [ ]:
df_train['datetime'] = pd.to_datetime(df_train['datetime'], format='%Y-%m-%d %H:%M:%S', errors='raise')
df_train.groupby(pd.Grouper(key='datetime', freq='D')).mean().head(20)

### train 과 test 컬럼 확인
- test 에는 casual, registered, count 가 없고, 
- casual + registered = count 이므로,
- test 의 count 를 예측하는 것이 목표임을 이해할 수 있음

> train 은 매달 1일부터 19일까지, test 는 매달 20일부터, 마지막일까지를 가지고 있다고 캐글 사이트에서 설명하고 있음
> https://www.kaggle.com/c/bike-sharing-demand/data

In [ ]:
df_train['datetime_timestamp'] = pd.to_datetime(df_train['datetime'], format='%Y-%m-%d %H:%M:%S')
df_train['datetime_timestamp'].dt.strftime('%Y-%m-%d').unique()

In [ ]:
df_test['datetime_timestamp'] = pd.to_datetime(df_test['datetime'], format='%Y-%m-%d %H:%M:%S')
df_test['datetime_timestamp'].dt.strftime('%Y-%m-%d').unique()

### 결측치 확인

In [ ]:
missingno.matrix(df_train)

In [ ]:
missingno.matrix(df_test)

### Understanding

In [ ]:
df_train = pd.read_csv('bikesharing/train.csv')
df_test = pd.read_csv('bikesharing/test.csv')
df_all = pd.concat((df_train, df_test)).reset_index(drop=True)

In [ ]:
df_train.shape

In [ ]:
df_test.shape

In [ ]:
df_train = df_all[:10886]
df_test = df_all[10886:]

In [ ]:
def split_df(df):
    return df[:10886], df[10886:]

In [ ]:
dt = pd.DatetimeIndex(df_all['datetime'])
df_all.set_index(dt, inplace=True)

df_all['date'] = dt.date
df_all['day'] = dt.day
df_all['month'] = dt.month
df_all['year'] = dt.year
df_all['hour'] = dt.hour
df_all['dow'] = dt.dayofweek

# 202502 업데이트: 최신 라이브러리에서는 weekofyear 가 지원되지 않음
# 기존 코드: df_all['woy'] = dt.weekofyear
df_all['woy'] = dt.isocalendar().week

In [ ]:
df_all.head()

- 본 머신러닝 문제의 성능 평가는 RMSLE(Root Mean Squared Log Error) 를 사용하므로,
- RMSLE 계산을 위해, 관련 예측해야 하는 값들을 RMSLE 공식에 맞게, 변환한 컬럼을 추가함
$$ RMSLE = \sqrt{\dfrac{\sum_{i=0}^N (log(y_i + 1) - log(\hat{y_i} + 1))^2 }{N}} $$ 

- 참고: 위의 log 는 자연 로그로, 자연 로그(natural logarithm)는 e를 밑으로 하는 로그를 의미함 

In [ ]:
df_all['casual_log'] = np.log(df_all['casual'] + 1)
df_all['registered_log'] = np.log(df_all['registered'] + 1)
df_all['count_log'] = np.log(df_all['count'] + 1)

### 타임 기반 EDA

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

df_train, df_test = split_df(df_all)

def visualize(column_name):
    # 202502 업데이트: sum() 메서드 사용시 데이터프레임이 숫자형 데이터 컬럼으로만 구성되어야 함
    # 이를 위해 ['count'] 에 대해서만 sum() 을 수행하도록 변경
    # 기존 코드
    # df_train_workingday_time = df_train.groupby(['workingday', column_name]).sum()
    # df_train_workingday_time = df_train_workingday_time.reset_index()
    df_train_workingday_time = df_train.groupby(['workingday', column_name])['count'].sum().reset_index()
    
    times_nonworkday = df_train_workingday_time[df_train_workingday_time['workingday'] == 0][column_name]
    counts_nonworkday = df_train_workingday_time[df_train_workingday_time['workingday'] == 0]['count']

    times_workday = df_train_workingday_time[df_train_workingday_time['workingday'] == 1][column_name]
    counts_workday = df_train_workingday_time[df_train_workingday_time['workingday'] == 1]['count']

    df_temp_workday = pd.concat([times_workday, counts_workday], axis=1, keys=[column_name, 'count'])
    df_temp_nonworkday = pd.concat([times_nonworkday, counts_nonworkday], axis=1, keys=[column_name, 'count'])
    
    fig = make_subplots(rows=1, cols=2, subplot_titles=("workday", "non-workday"))
    fig.add_trace( 
        go.Bar( x=df_temp_workday[column_name], y=df_temp_workday['count'], text=df_temp_workday['count'], textposition='auto', texttemplate='%{text:.0f}' ),
        row=1, col=1
    )
    fig.add_trace( 
        go.Bar( x=df_temp_nonworkday[column_name], y=df_temp_nonworkday['count'], text=df_temp_nonworkday['count'], textposition='auto', texttemplate='%{text:.0f}' ),
        row=1, col=2        
    )
    fig.update_layout(showlegend=False)
    fig.show()

In [ ]:
visualize('year')

In [ ]:
visualize('month')

In [ ]:
visualize('day')

In [ ]:
visualize('dow')

In [ ]:
visualize('woy')

In [ ]:
visualize('hour')

> 시간대별은 의미가 있어 보임 (피크 시간대가 있어 보임)

### 시간대 분석

In [ ]:
df_train, df_test = split_df(df_all)

# 202502 업데이트: sum() 메서드 사용시 데이터프레임이 숫자형 데이터 컬럼으로만 구성되어야 함
# 이를 위해 ['count'] 에 대해서만 sum() 을 수행하도록 변경
# 기존 코드
# df_train_workingday_hour = df_train.groupby(['workingday', 'hour']).sum()
# df_train_workingday_hour = df_train_workingday_hour.reset_index()
df_train_workingday_hour = df_train.groupby(['workingday', 'hour'])['count'].sum().reset_index()

hours_nonworkday = df_train_workingday_hour[df_train_workingday_hour['workingday'] == 0]['hour']
counts_nonworkday = df_train_workingday_hour[df_train_workingday_hour['workingday'] == 0]['count']

hours_workday = df_train_workingday_hour[df_train_workingday_hour['workingday'] == 1]['hour']
counts_workday = df_train_workingday_hour[df_train_workingday_hour['workingday'] == 1]['count']

df_temp_workday = pd.concat([hours_workday, counts_workday], axis=1, keys=['hour', 'count'])
df_temp_nonworkday = pd.concat([hours_nonworkday, counts_nonworkday], axis=1, keys=['hour', 'count'])

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(
    go.Bar( x=df_temp_workday['hour'], y=df_temp_workday['count'], text=df_temp_workday['count'], textposition='auto', texttemplate='%{text:.0f}' )
)
fig.update_layout({ 
        "title": { "text": "<b>workday</b>" },    
        "xaxis": { "showticklabels":True, "dtick": "1" },
        "template":'ggplot2'            
})
fig.show()

fig = go.Figure()
fig.add_trace(
    go.Bar( x=df_temp_nonworkday['hour'], y=df_temp_nonworkday['count'], text=df_temp_nonworkday['count'], textposition='auto', texttemplate='%{text:.0f}' )
)
fig.update_layout({
        "title": { "text": "<b>non-workday</b>" },
        "xaxis": { "showticklabels":True, "dtick": "1" },
        "template":'seaborn'            
})
fig.show()

- workday peak time: 8, 17 ~ 18
- non-workday peak time: 10 ~ 19

In [ ]:
def func(df_data):
    if df_data['workingday'] == 1:
        if (df_data['hour'] == 8) or (df_data['hour'] >= 17 and df_data['hour'] <= 18):
            return 1
    else:
        if (df_data['hour'] >= 10 and df_data['hour'] <= 19):
            return 1
    return 0

# 0 or ‘index’: 각 컬럼에 함수 적용, 1 or ‘columns’: 각 행에 함수 적용
df_all['peak'] = df_all.apply(func, axis=1)

### 상관관계 분석

In [ ]:
df_train, df_test = split_df(df_all)

In [ ]:
df_train.columns

In [ ]:
df_train_corr = df_train[['season', 'holiday', 'workingday', 'weather', 'temp', 'atemp', 'humidity', 'windspeed', 'casual', 'registered', 'count']]

In [ ]:
import plotly.graph_objects as go
import numpy as np

# df_train_corr로부터 상관계수 행렬 계산
corr_matrix = df_train_corr.corr()

# 각 셀에 표시할 상관관계 값 (소수점 둘째자리)
text_values = np.round(corr_matrix.values, 2)

fig = go.Figure(data=go.Heatmap(
    z=corr_matrix.values,
    x=corr_matrix.columns,
    y=corr_matrix.index,
    colorscale='Blues',   # 파란 계통 컬러 스케일 사용
    zmin=-1,              # 컬러바 범위 -1부터
    zmax=1,               # 컬러바 범위 1까지
    text=text_values,     # 셀에 표시할 텍스트 값
    texttemplate="%{text}"  # 텍스트 포맷 설정

))

fig.update_layout(
    title="Correlation Heatmap",
    xaxis_title="Variables",
    yaxis_title="Variables"
)

fig.show()

### 온도, 풍속, 습도 분석

In [ ]:
df_all.loc[ df_all['windspeed'] <= 5, 'windspeed_category'] = 0
df_all.loc[(df_all['windspeed'] > 5) & (df_all['windspeed'] <= 10), 'windspeed_category'] = 1
df_all.loc[(df_all['windspeed'] > 10) & (df_all['windspeed'] <= 15), 'windspeed_category'] = 2
df_all.loc[(df_all['windspeed'] > 15) & (df_all['windspeed'] <= 20), 'windspeed_category'] = 3
df_all.loc[(df_all['windspeed'] > 20) & (df_all['windspeed'] <= 25), 'windspeed_category'] = 4
df_all.loc[(df_all['windspeed'] > 25) & (df_all['windspeed'] <= 30), 'windspeed_category'] = 5
df_all.loc[(df_all['windspeed'] > 30) & (df_all['windspeed'] <= 35), 'windspeed_category'] = 6
df_all.loc[(df_all['windspeed'] > 35) & (df_all['windspeed'] <= 40), 'windspeed_category'] = 7
df_all.loc[(df_all['windspeed'] > 35) & (df_all['windspeed'] <= 40), 'windspeed_category'] = 8
df_all.loc[(df_all['windspeed'] > 40) & (df_all['windspeed'] <= 45), 'windspeed_category'] = 9
df_all.loc[(df_all['windspeed'] > 45) & (df_all['windspeed'] <= 50), 'windspeed_category'] = 10
df_all.loc[ df_all['windspeed'] > 50, 'windspeed_category'] = 11

In [ ]:
# 202502 업데이트: sum() 메서드 사용시 데이터프레임이 숫자형 데이터 컬럼으로만 구성되어야 함
# 이를 위해 ['count'] 에 대해서만 sum() 을 수행하도록 변경
# 기존 코드: df_temp_count = df_all.groupby('windspeed_category').sum()
df_temp_count = df_all.groupby('windspeed_category')['count'].sum().reset_index()

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Bar( x=df_temp_count.index, y=df_temp_count['count'], text=df_temp_count['count'], textposition='auto', texttemplate='%{text:.0f}' )
)
fig.update_layout({ 
        "title": { "text": "<b>windspeed VS count </b>" },    
        "xaxis": { "showticklabels":True, "dtick": "1" },
        "template":'ggplot2'            
})
fig.show()

In [ ]:
df_all['temp'].describe()

In [ ]:
df_all.loc[ df_all['temp'] <= 5, 'temp_category'] = 0
df_all.loc[(df_all['temp'] > 5) & (df_all['temp'] <= 10), 'temp_category'] = 1
df_all.loc[(df_all['temp'] > 10) & (df_all['temp'] <= 15), 'temp_category'] = 2
df_all.loc[(df_all['temp'] > 15) & (df_all['temp'] <= 20), 'temp_category'] = 3
df_all.loc[(df_all['temp'] > 20) & (df_all['temp'] <= 25), 'temp_category'] = 4
df_all.loc[(df_all['temp'] > 25) & (df_all['temp'] <= 30), 'temp_category'] = 5
df_all.loc[(df_all['temp'] > 30) & (df_all['temp'] <= 35), 'temp_category'] = 6
df_all.loc[(df_all['temp'] > 35) & (df_all['temp'] <= 40), 'temp_category'] = 7
df_all.loc[ df_all['temp'] > 40, 'temp_category'] = 8

In [ ]:
# 202502 업데이트: sum() 메서드 사용시 데이터프레임이 숫자형 데이터 컬럼으로만 구성되어야 함
# 이를 위해 ['count'] 에 대해서만 sum() 을 수행하도록 변경
# 기존 코드: df_temp_count = df_all.groupby('temp_category').sum()
df_temp_count = df_all.groupby('temp_category')['count'].sum().reset_index()

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Bar( x=df_temp_count.index, y=df_temp_count['count'], text=df_temp_count['count'], textposition='auto', texttemplate='%{text:.0f}' )
)
fig.update_layout({ 
        "title": { "text": "<b>temperature VS count </b>" },    
        "xaxis": { "showticklabels":True, "dtick": "1" },
        "template":'ggplot2'            
})
fig.show()

In [ ]:
df_all['humidity'].describe()

In [ ]:
df_all.loc[ df_all['humidity'] <= 10, 'humidity_category'] = 0
df_all.loc[(df_all['humidity'] > 10) & (df_all['humidity'] <= 20), 'humidity_category'] = 1
df_all.loc[(df_all['humidity'] > 20) & (df_all['humidity'] <= 30), 'humidity_category'] = 2
df_all.loc[(df_all['humidity'] > 30) & (df_all['humidity'] <= 40), 'humidity_category'] = 3
df_all.loc[(df_all['humidity'] > 40) & (df_all['humidity'] <= 50), 'humidity_category'] = 4
df_all.loc[(df_all['humidity'] > 50) & (df_all['humidity'] <= 60), 'humidity_category'] = 5
df_all.loc[(df_all['humidity'] > 60) & (df_all['humidity'] <= 70), 'humidity_category'] = 6
df_all.loc[(df_all['humidity'] > 70) & (df_all['humidity'] <= 80), 'humidity_category'] = 7
df_all.loc[(df_all['humidity'] > 80) & (df_all['humidity'] <= 90), 'humidity_category'] = 8
df_all.loc[ df_all['humidity'] > 90, 'humidity_category'] = 9

In [ ]:
# 202502 업데이트: sum() 메서드 사용시 데이터프레임이 숫자형 데이터 컬럼으로만 구성되어야 함
# 이를 위해 ['count'] 에 대해서만 sum() 을 수행하도록 변경
# 기존 코드: df_temp_count = df_all.groupby('humidity_category').sum()
df_temp_count = df_all.groupby('humidity_category')['count'].sum().reset_index()

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Bar( x=df_temp_count.index, y=df_temp_count['count'], text=df_temp_count['count'], textposition='auto', texttemplate='%{text:.0f}' )
)
fig.update_layout({ 
        "title": { "text": "<b>humidity VS count </b>" },    
        "xaxis": { "showticklabels":True, "dtick": "1" },
        "template":'ggplot2'            
})
fig.show()

In [ ]:
df_all.head()

In [ ]:
# 202502 업데이트: sum() 메서드 사용시 데이터프레임이 숫자형 데이터 컬럼으로만 구성되어야 함
# 기존 코드: df_temp_count = df_all.groupby(['workingday', 'humidity_category']).sum()
# 숫자형 데이터 컬럼 필터링 방법
# select_dtypes(): 특정 데이터 타입의 열만 선택하는 pandas 메서드 (include=[np.number]: 숫자 타입의 열만 포함)
df_numeric = df_all.select_dtypes(include=[np.number])  # numpy 라이브러리가 필요
df_temp_count = df_numeric.groupby(['workingday', 'humidity_category']).sum()

In [ ]:
df_temp_count.head()

In [ ]:
df_temp_count.loc[0]['count']

### 복합 분석

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Bar( x=df_temp_count.loc[0].index, y=df_temp_count.loc[0]['count'], name="non-workingday" )
)
fig.add_trace(
    go.Bar( x=df_temp_count.loc[1].index, y=df_temp_count.loc[1]['count'], name="workingday" )
)

fig.update_layout({ 
        "title": { "text": "<b>humidity & workingday VS count </b>" },    
        "xaxis": { "showticklabels":True, "dtick": "1" },
        "barmode": "stack",
        "template":'ggplot2'            
})
fig.show()

In [ ]:
# 202502 업데이트: sum() 메서드 사용시 데이터프레임이 숫자형 데이터 컬럼으로만 구성되어야 함
# 기존 코드: df_temp_count = df_all.groupby(['temp_category', 'windspeed_category']).sum()
# 숫자형 데이터 컬럼 필터링 방법
# select_dtypes(): 특정 데이터 타입의 열만 선택하는 pandas 메서드 (include=[np.number]: 숫자 타입의 열만 포함)
df_numeric = df_all.select_dtypes(include=[np.number])  # numpy 라이브러리가 필요
df_temp_count = df_numeric.groupby(['temp_category', 'windspeed_category']).sum()

In [ ]:
df_temp_count['count']

In [ ]:
df_temp_count.loc[0].index

In [ ]:
df_temp_count.loc[0]['count']

In [ ]:
fig = go.Figure()


fig.add_trace(
    go.Bar( x=df_temp_count.loc[0].index, y=df_temp_count.loc[0]['count'], name="temperature " + str(0) )
)
fig.add_trace(
    go.Bar( x=df_temp_count.loc[1].index, y=df_temp_count.loc[1]['count'], name="temperature " + str(1) )
)

fig.update_layout({ 
        "title": { "text": "<b>temperature & windspeed VS count </b>" },    
        "xaxis": { "showticklabels":True, "dtick": "1", "title": "windspeed" },
        "barmode": "stack",
        "template":'ggplot2'            
})
fig.show()

In [ ]:
fig = go.Figure()

for index in range(9):
    fig.add_trace(
        go.Bar( x=df_temp_count.loc[index].index, y=df_temp_count.loc[index]['count'], name="temperature " + str(index) )
    )

fig.update_layout({ 
        "title": { "text": "<b>temperature & windspeed VS count </b>" },    
        "xaxis": { "showticklabels":True, "dtick": "1", "title": "windspeed" },
        "barmode": "stack",
        "template":'ggplot2'            
})
fig.show()

In [ ]:
# 202502 업데이트: sum() 메서드 사용시 데이터프레임이 숫자형 데이터 컬럼으로만 구성되어야 함
# 기존 코드: df_temp_count = df_all.groupby(['temp_category', 'humidity_category']).sum()
# 숫자형 데이터 컬럼 필터링 방법
# select_dtypes(): 특정 데이터 타입의 열만 선택하는 pandas 메서드 (include=[np.number]: 숫자 타입의 열만 포함)
df_numeric = df_all.select_dtypes(include=[np.number])  # numpy 라이브러리가 필요
df_temp_count = df_numeric.groupby(['temp_category', 'humidity_category']).sum()

In [ ]:
fig = go.Figure()

for index in range(9):
    fig.add_trace(
        go.Bar( x=df_temp_count.loc[index].index, y=df_temp_count.loc[index]['count'], name="temperature " + str(index) )
    )

fig.update_layout({ 
        "title": { "text": "<b>temperature & humidity VS count </b>" },    
        "xaxis": { "showticklabels":True, "dtick": "1" },
        "barmode": "stack",
        "template":'ggplot2'            
})
fig.show()

In [ ]:
df_train, df_test = split_df(df_all)

In [ ]:
# 202502 업데이트: sum() 메서드 사용시 데이터프레임이 숫자형 데이터 컬럼으로만 구성되어야 함
# 기존 코드: df_temp_count = df_train.groupby(['weather']).sum()
# 숫자형 데이터 컬럼 필터링 방법
# select_dtypes(): 특정 데이터 타입의 열만 선택하는 pandas 메서드 (include=[np.number]: 숫자 타입의 열만 포함)
df_numeric = df_all.select_dtypes(include=[np.number])  # numpy 라이브러리가 필요
df_temp_count = df_numeric.groupby(['weather']).sum()

#### weather:
 - 1: Clear, Few clouds, Partly cloudy, Partly cloudy
 - 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
 - 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
 - 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Bar( x=df_temp_count.index, y=df_temp_count['count'], text=df_temp_count['count'], textposition='auto', texttemplate='%{text:.0f}' )
)
fig.update_layout({ 
        "title": { "text": "<b>weather VS count </b>" },    
        "xaxis": { "showticklabels":True, "dtick": "1" },
        "template":'ggplot2'            
})
fig.show()

In [ ]:
# 202502 업데이트: sum() 메서드 사용시 데이터프레임이 숫자형 데이터 컬럼으로만 구성되어야 함
# 기존 코드: df_temp_count = df_train.groupby(['season']).sum()
# 숫자형 데이터 컬럼 필터링 방법
# select_dtypes(): 특정 데이터 타입의 열만 선택하는 pandas 메서드 (include=[np.number]: 숫자 타입의 열만 포함)
df_numeric = df_train.select_dtypes(include=[np.number])  # numpy 라이브러리가 필요
df_temp_count = df_numeric.groupby(['season']).sum()

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Bar( x=df_temp_count.index, y=df_temp_count['count'], text=df_temp_count['count'], textposition='auto', texttemplate='%{text:.0f}' )
)
fig.update_layout({ 
        "title": { "text": "<b>season VS count </b>" },    
        "xaxis": { "showticklabels":True, "dtick": "1" },
        "template":'ggplot2'            
})
fig.show()

In [ ]:
# 202502 업데이트: sum() 메서드 사용시 데이터프레임이 숫자형 데이터 컬럼으로만 구성되어야 함
# 기존 코드: df_temp_count = df_train.groupby(['temp_category', 'weather']).sum()
# 숫자형 데이터 컬럼 필터링 방법
# select_dtypes(): 특정 데이터 타입의 열만 선택하는 pandas 메서드 (include=[np.number]: 숫자 타입의 열만 포함)
df_numeric = df_train.select_dtypes(include=[np.number])  # numpy 라이브러리가 필요
df_temp_count = df_numeric.groupby(['temp_category', 'weather']).sum()

In [ ]:
fig = go.Figure()

for index in range(9):
    fig.add_trace(
        go.Bar( x=df_temp_count.loc[index].index, y=df_temp_count.loc[index]['count'], name="temp " + str(index) )
    )

fig.update_layout({ 
        "title": { "text": "<b>weather & temperature VS count </b>" },    
        "xaxis": { "showticklabels":True, "dtick": "1" },
        "barmode": "stack",
        "template":'ggplot2'            
})
fig.show()

### 날짜 분석

In [ ]:
df_all.head(5)

In [ ]:
df_all.tail(5)

<div class="alert alert-block" style="border: 2px solid #1976D2;background-color:#E3F2FD;padding:5px;font-size:0.9em;">
본 자료는 저작권법 제25조 2항에 의해 보호를 받습니다. 본 컨텐츠 및 컨텐츠 일부 문구등을 외부에 공개, 게시하지 말아주세요.<br>
본 강의를 잘 정리하면, 데이터 분석과 데이터 과학(머신러닝, 인공지능) 모두 가능합니다!<br>
<b><a href="https://school.fun-coding.org/">잔재미코딩</a> 에서 본 강의 기반 최적화된 로드맵도 확인하실 수 있습니다</b>
</div>